# Nhập khẩu thư viện


In [1]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models import resnet50
import warnings
from PIL import Image, ImageFile
import logging
from datetime import datetime

#  Cấu hình môi trường để xử lý các vấn đề thường gặp khi làm việc với dữ liệu ảnh trong thực tế



- PIL (Python Imaging Library) load được các file ảnh bị *truncated* (cắt cụt/không đầy đủ).

   PIL sẽ cố gắng load phần dữ liệu còn lại của ảnh.

   Phần bị thiếu sẽ được fill bằng màu đen hoặc màu mặc định.

   Giúp pipeline training không bị gián đoạn

- Tắt các warning messages thuộc loại UserWarning.

    Tránh spam, giúp log hiển thị log quan trọng trong training.

- Thiết lập hệ thống logging để theo dõi và ghi lại các sự kiện trong chương trình

In [2]:
# Cấu hình PIL để xử lý file ảnh bị truncated
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Tắt các warning messages thuộc loại UserWarning
warnings.filterwarnings("ignore", category=UserWarning)

# Thiết lập logging để theo dõi các file bị lỗi, có trả về file log :)
logging.basicConfig(
    filename="/kaggle/working/training.log",
    level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s'
)

# Các class

- SafeTransform: Tiền xử lý transform để tránh lỗi error handling
- DogCatDataset: Biến folder ảnh thô thành dataset sạch để train model
 + Tìm và gán nhãn
 + Lọc ảnh hợp lệ
 + Load ảnh khi cần
- DogCatClassifier: mô hình neural network để phân loại ảnh chó/mèo

### Class SafeTransform - tiền xử lý ảnh khỏi lỗi error handling

- Transform = "dịch thuật" ảnh từ dạng con người hiểu sang dạng AI hiểu, bằng:
 + ToPILImage,
 + ToTensor,
 + Normalize(chuẩn hoá theo cùng 1 khuân)

In [3]:
#Preprocessing transform với error handling
class SafeTransform:
    """
        Transform = "dịch thuật" ảnh từ dạng con người hiểu sang dạng AI hiểu

        Logic:
            Ảnh gốc (.jpg file)
                ↓ transform ToPILImage, ToTensor, Normalize(Chuẩn hóa theo ImageNet)
            Tensor (3, 224, 224) với giá trị [-2, +2]

        Hàm:
            - init : định nghĩa transform
            - call : thực hiện transform
    """

    def __init__(self):
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
        ])

    def __call__(self, image):
        try:
            return self.transform(image)
        except Exception as e:
            logging.error(f"Transform error: {str(e)}")
            # Trả về tensor mặc định
            return torch.zeros(3, 224, 224)

### Class Dataset

- Vì cách thức tổ chức thư mục theo kiểu:
 + Pets/
    + Cat/
        - cat1.jpg
    + Dog/
        - dog1.jpg

  nên thực hiện for đọc vào trong folder  
- Dán nhãn DOG = 1, CAT = 0
- Thực hiện kiểm tra đuôi (extension) và metadata của ảnh.
 - Kiểm tra metadata bằng def _is_valid_image
 - Kiểm tra ảnh với error handing bằng def _load_image_safely
 - Build time: _is_valid_image() → lọc file tốt.

   Training time: _load_image_safely() → load ảnh đã lọc

   

In [4]:
class DogCatDataset(Dataset):
    def __init__(self, root_folder, transform=None):
        self.image_paths = []
        self.labels = []
        self.transform = transform
        self.skipped_files = 0
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']

        for sub_folder in os.listdir(root_folder):
            sub_folder_path = os.path.join(root_folder, sub_folder)

            if os.path.isdir(sub_folder_path):
                if "dog" in sub_folder.lower():
                    label = 1
                elif "cat" in sub_folder.lower():
                    label = 0
                else:
                    continue # Bỏ vòng for bên dưới!

                for file_name in os.listdir(sub_folder_path):
                    if any(file_name.lower().endswith(ext) for ext in image_extensions):
                        image_path = os.path.join(sub_folder_path, file_name)

                        # Kiểm tra tính hợp lệ của file ảnh trong metadata
                        if self.is_valid_image(image_path):
                            self.image_paths.append(image_path)
                            self.labels.append(label)
                        else:
                            self.skipped_files += 1
                            logging.warning(f"Skipping invalid image: {image_path}")
        logging.info(f"Total valid images loaded: {len(self.image_paths)}")
        logging.info(f"Total skipped files: {self.skipped_files}")

    def is_valid_image(self, image_path):
        """Kiểm tra tính hợp lệ của file ảnh"""
        try:
            # Kiểm tra với OpenCV
            img = cv2.imread(image_path)
            if img is None:
                return False
            # Kiểm tra kích thước hợp lệ
            if img.shape[0] < 32 or img.shape[1] < 32:
                return False
            # Kiểm tra với PIL để catch lỗi JPEG hỏng
            with Image.open(image_path) as img_pil:
                img_pil.verify()
            # Kiểm tra với PIL để đảm bảo có thể load được
            with Image.open(image_path) as img_pil:
                img_pil.load()
            return True

        except (OSError, IOError, cv2.error) as e:
            logging.warning(f"Skipping invalid image: {image_path} - Error: {str(e)}")
            return False

        except Exception as e:
            logging.warning(f"Skipping invalid image: {image_path} - Error: {str(e)}")
            return False

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        try:
            # Đọc ảnh với error handling
            image = self._load_image_safely(self.image_paths[idx])
            if self.transform:
                image = self.transform(image)
            label = self.labels[idx]
            return image, label

        except Exception as e:
            logging.error(f"Error loading image: {self.image_paths[idx]} - Error: {str(e)}")
            # Trả về ảnh mặc định hoặc tensor rỗng
            if self.transform:
                default_image = np.zeros(3, 224, 224)
                image = self.transform(default_image)
            else:
                image = torch.zeros(3, 224, 224)
            label = self.labels[idx]
            return image, label

    def _load_image_safely(self, image_path):
        """Đọc ảnh một cách an toàn với nhiều phương pháp fallback"""
        # Thử đọc với OpenCV
        try:
            image = cv2.imread(image_path)
            if image is not None:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, (224, 224))
                return image
        except Exception as e:
            pass  # Nếu lỗi xảy ra, tiếp tục với PIL

        # Thử đọc với PIL
        try:
            with Image.open(image_path) as image:
                image = image.convert('RGB')
                image = image.resize((224, 224))
                image = np.array(image)
                return image
        except Exception as e:
            logging.error(f"Error loading image: {image_path} - Error: {str(e)}")
            return np.zeros((224, 224, 3), dtype=np.uint8)

### Class DataClassifier

 + Lấy ResNet50 đã pre-trained (biết nhận diện 1000 objects)
 + Thay lớp cuối từ 1000 classes → 2 classes (chó/mèo)
 + Forward: Nhận ảnh tensor → trả về điểm số [cat_score, dog_score]

In [5]:
class DogCatClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(DogCatClassifier, self).__init__()
        self.resnet = resnet50(pretrained=True)
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# Training model

In [6]:
def train_model(model, train_loader, val_loader, device, epochs=10):
    criterion = nn.CrossEntropyLoss(ignore_index=-1)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.to(device)

    for epoch in range(epochs):

        # Training
        model.train() # Bật tính năng training
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        train_batch_errors = 0

        for batch_idx, (images, labels) in enumerate(train_loader):
            try:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                train_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()

            except Exception as e:
                logging.error(f"Error during training: Batch {batch_idx} - Error: {str(e)}")
                train_batch_errors += 1
                continue

        train_accuracy = 100 * train_correct / train_total if train_total > 0 else 0
        avg_train_loss = train_loss / (len(train_loader) - train_batch_errors) if (len(train_loader) - train_batch_errors) > 0 else 0
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_train_loss: .4f}, Accuracy: {train_accuracy: .4f}%")

        if train_batch_errors > 0:
            print(f"Train batch errors: {train_batch_errors}")

        #Validation
        model.eval() # Bật tính năng validate
        val_correct = 0
        val_total = 0
        val_batch_errors = 0

        with torch.no_grad():
            for batch_idx, (images, labels) in enumerate(val_loader):
                try:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    _, predicted = torch.max(outputs.data, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()

                except Exception as e:
                    logging.error(f"Error during validation: Batch {batch_idx} - Error: {str(e)}")
                    val_batch_errors += 1
                    continue

        val_accuracy = 100 * val_correct / val_total if val_total > 0 else 0
        print(f"Validation Accuracy: {val_accuracy: .4f}%")
        if val_batch_errors > 0:
            print(f"Validation batch errors: {val_batch_errors}")

# Kiểm tra model bằng 1 dữ liệu bất kỳ từ ngoài

In [7]:
def predict_image(image_path, model, device):
    try:
        # Load ảnh an toàn
        dataset_temp = DogCatDataset.__new__(DogCatDataset)
        image = dataset_temp._load_image_safely(image_path)

        transform = SafeTransform()
        image = transform(image)
        image = image.unsqueeze(0).to(device)

        model.eval()
        with torch.no_grad():
            output = model(image)
            probabilities = torch.softmax(output, dim=1)
            confidence, predicted = torch.max(probabilities, 1)

        if predicted.item() == 0:
            return "Đây là mèo!", confidence.item()
        else:
            return "Đây là chó!", confidence.item()

    except Exception as e:
        logging.error(f"Error during prediction: {str(e)}")
        print("Không thể dự đoán ảnh này.")
        return "Error", 0.0

# Main

- Kiểm tra device sẽ chạy code là GPU với cuda hay CPU
- Cấu hình batch và epochs
- Đọc thư mục chứa data train và validate
- Thiết lập dataset và datasetloader
  + Cần xử lý Error handling (xử lý lỗi) - cách chương trình phản ứng khi gặp lỗi thay vì crash (dừng hẳn)= > dùng ***try***
-

In [ ]:
if __name__ == "__main__":

    #kiểm tra thiết bị sẽ chạy code là GPU hay CPU
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    logging.info(f"Sử dụng device: {device}")

    # cấu hình batch và epochs
    batch_size = 256 # 64, 256, 512
    epochs = 20 # 10, 20, 25, 50

    # Đọc thư mục chứa data gốc
    root_folder = "/kaggle/input/mynewnewpet/Pets"

    # Dataset and DataLoader với error handling
    try:
        print("Đang tải dataset...")

        #Transform = biến đổi/xử lý ảnh để chuẩn bị cho model deep learning.
        transform = SafeTransform()
        dataset = DogCatDataset(root_folder, transform=transform)

        if len(dataset) == 0:
            raise ValueError("Không tìm thấy ảnh hợp lệ nào trong dataset!")

        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

                # num_workers=0 để tránh lỗi => Chỉ skip file lỗi, không crash toàn bộ
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

        # creat model
        model = DogCatClassifier()
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
        model.to(device)

        # train model
        print("Đang train model...")
        train_model(model, train_loader, val_loader, device, epochs)

        # save model
        model_path = f"model_resnet50_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pth"
        torch.save(model.state_dict(), model_path)
        print(f"Model đã được lưu tại: {model_path}")

        # Kiểm tra model bằng 1 tấm ảnh ở ngoài mô hình (ảnh bất kỳ)
        test_image_path = "/kaggle/input/funny-dog-h-jpg/Funny_Dog_H.jpg"
        if os.path.exists(test_image_path):
            test_result, test_confidence = predict_image(test_image_path, model, device)
            print(f"Kết quả dự đoán: {test_result}, độ tin cậy: {test_confidence}")
        else:
            print(f"File test {test_image_path} không tồn tại!")

    except Exception as e:
        logging.error(f"Error during dataset loading: {str(e)}")
        print(f"Lỗi trong quá trình tải dataset: {str(e)}")